In [2]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections
from collections import Counter

In [6]:
def collect_metrics(root_dir):

    depth = 2

    dict_res_roberta = dict()
    dict_res_camem = dict()
    dict_res_xlm = dict()
    
    for subdir, dirs, files in os.walk(root_dir):
        if subdir[len(root_dir):].count(os.sep) < depth:
            for file_ in files:
                if file_ == "xai_metrics.json":
                    with open(subdir + "/" + file_) as f:
                        d = json.load(f)
                        log_odd = d["log_odd"]
                        anti_log_odd = d["anti_log_odd"]
                        comp = d["comp"]
                        suff = d["suff"]
                        
                    model_type = subdir.split(os.sep)[-1].split("_")[0]
                    if model_type == "roberta":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_roberta[dataset_name] = {"log odd": log_odd, "anti log odd": anti_log_odd, "comp": comp, "suff": suff}
                    elif model_type == "camem":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_camem[dataset_name] = {"log odd": log_odd, "anti log odd": anti_log_odd, "comp": comp, "suff": suff}
                    elif model_type == "xlm":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_xlm[dataset_name] = {"log odd": log_odd, "anti log odd": anti_log_odd, "comp": comp, "suff": suff}

    return dict_res_roberta, dict_res_camem, dict_res_xlm

def _average_dicts(dicts):
    avg_dict = collections.defaultdict(lambda: 0)
    
    for i, d in enumerate(dicts):
        for k, v in d.items():
            avg_dict[k] += v
        
    for k, v in avg_dict.items():
        avg_dict[k] /= (i+1)
    
    return avg_dict

def average_over_users(res):
    averaged_en = collections.defaultdict(list)
    averaged_it = collections.defaultdict(list)

    for k, v in res.items():
        if "en" in k and not ("nf" in k):
            averaged_en[" ".join(k.split(" ")[0:2])].append(v)
        elif "it" in k and not ("nf" in k):
            averaged_it[" ".join(k.split(" ")[0:2])].append(v)
    
    result_en = dict()
    result_it = dict()
            
    for k, v in averaged_en.items():
        result_en["avg en " + k] = _average_dicts(v)
        
    for k, v in averaged_it.items():
        result_it["avg it " + k] = _average_dicts(v)
                        
    return result_en, result_it

def process_to_present(res, column_width="1.1cm", plot_transpose=True):
    df_res = pd.DataFrame.from_dict(res).T
    # df_res = df_res.rename(columns=mapping_columns_names)
    if not plot_transpose:
        df_res = df_res.T
    s = df_res.style
    s.format(na_rep='MISS', precision=2)
    print(s.to_latex(column_format='l'+('p{'+f'{column_width}'+'}')*len(df_res.columns)))

In [7]:
dict_res_roberta, dict_res_camem, dict_res_xlm = collect_metrics("../output/sst2")

## ROBERTA

In [14]:
avg, _ = average_over_users(dict_res_roberta)
roberta_results = {**dict_res_roberta, ** avg}

In [15]:
pd.DataFrame(roberta_results).T

,log odd,anti log odd,comp,suff
np f en6,-0.780508,-0.351081,0.307785,0.017361
p f en98,-0.461949,-0.608122,0.221173,0.260342
p f en6,-0.645478,-0.441514,0.220512,0.300342
p f en83,-0.770531,-0.500421,0.270367,0.218905
np f en57,-0.816791,-1.390459,0.376330,-0.015842
p f en49,-0.679228,-0.657288,0.247378,0.283853
p f en57,-0.741071,-0.665086,0.264203,0.273420
np f en83,-0.668076,-1.300311,0.324050,0.035894
np f en49,-1.089605,-1.219152,0.392459,-0.002587
np f en98,-0.924336,-1.234849,0.396752,0.005183


In [18]:
process_to_present(dict_res_roberta)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & log odd & anti log odd & comp & suff \\
np f en6 & -0.78 & -0.35 & 0.31 & 0.02 \\
p f en98 & -0.46 & -0.61 & 0.22 & 0.26 \\
p f en6 & -0.65 & -0.44 & 0.22 & 0.30 \\
p f en83 & -0.77 & -0.50 & 0.27 & 0.22 \\
np f en57 & -0.82 & -1.39 & 0.38 & -0.02 \\
p f en49 & -0.68 & -0.66 & 0.25 & 0.28 \\
p f en57 & -0.74 & -0.67 & 0.26 & 0.27 \\
np f en83 & -0.67 & -1.30 & 0.32 & 0.04 \\
np f en49 & -1.09 & -1.22 & 0.39 & -0.00 \\
np f en98 & -0.92 & -1.23 & 0.40 & 0.01 \\
np nf en & -0.62 & -1.59 & 0.32 & 0.04 \\
p nf en & -0.78 & -0.55 & 0.24 & 0.26 \\
\end{tabular}



## XLM

In [16]:
avg, _ = average_over_users(dict_res_xlm)
xlm_results = {**dict_res_xlm, ** avg}

In [17]:
pd.DataFrame(xlm_results).T

,log odd,anti log odd,comp,suff
p nf en,-0.375523,-0.513320,0.123455,0.291769
np f en49,-1.356566,0.020259,0.425707,-0.058578
p f en83,-0.203729,-0.362042,0.044817,0.108043
p f en49,-0.623421,-0.553534,0.154762,0.319799
np f en83,-1.371232,-0.539930,0.409072,-0.053920
p f en98,-0.289122,-0.381917,0.123982,0.269300
np f en98,-1.195418,-0.076866,0.307537,0.052349
np f en6,-1.223319,-0.053538,0.330342,0.028008
np nf en,-0.785597,-1.182129,0.312279,0.109514
p f en57,-0.130832,-0.307203,0.069295,0.508367


In [19]:
process_to_present(dict_res_xlm)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & log odd & anti log odd & comp & suff \\
p nf en & -0.38 & -0.51 & 0.12 & 0.29 \\
np f en49 & -1.36 & 0.02 & 0.43 & -0.06 \\
p f en83 & -0.20 & -0.36 & 0.04 & 0.11 \\
p f en49 & -0.62 & -0.55 & 0.15 & 0.32 \\
np f en83 & -1.37 & -0.54 & 0.41 & -0.05 \\
p f en98 & -0.29 & -0.38 & 0.12 & 0.27 \\
np f en98 & -1.20 & -0.08 & 0.31 & 0.05 \\
np f en6 & -1.22 & -0.05 & 0.33 & 0.03 \\
np nf en & -0.79 & -1.18 & 0.31 & 0.11 \\
p f en57 & -0.13 & -0.31 & 0.07 & 0.51 \\
p f en6 & -0.21 & -0.27 & 0.08 & 0.34 \\
np f en57 & -1.22 & -0.06 & 0.34 & -0.01 \\
\end{tabular}

